In [ ]:
# GoEmotions DeBERTa-v3-large Multi-Label Classification
# Using local caching for fast, offline training

!nvidia-smi

# Install packages with conflict resolution
!pip install --upgrade pip --root-user-action=ignore
!pip install torch==2.3.1+cu118 torchvision==0.18.1+cu118 torchaudio==2.3.1+cu118 --index-url https://download.pytorch.org/whl/cu118 --root-user-action=ignore
!pip install transformers accelerate datasets evaluate scikit-learn tensorboard pyarrow tiktoken --root-user-action=ignore

# Change to the project root directory
import os
os.chdir('/home/user/goemotions-deberta')
print(f"📁 Current directory: {os.getcwd()}")

# Setup local caching (run this first time only)
print("🚀 Setting up local cache...")
!python3 scripts/setup_local_cache.py


Wed Sep  3 11:24:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.08             Driver Version: 550.127.08     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        On  |   00000000:C1:00.0 Off |                  N/A |
| 30%   26C    P8             38W /  350W |       2MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Training with DeBERTa-v3-large using local cache
!accelerate launch --num_processes=2 --mixed_precision=fp16 \
scripts/train_deberta_local.py \
--output_dir "./outputs/deberta" \
--model_type "deberta-v3-large" \
--per_device_train_batch_size 8 --per_device_eval_batch_size 16 \
--gradient_accumulation_steps 4 \
--num_train_epochs 3 \
--learning_rate 1e-5 --lr_scheduler_type cosine --warmup_ratio 0.1 \
--weight_decay 0.01 --fp16 --tf32 --gradient_checkpointing


/usr/bin/python3: can't open file '/home/user/goemotions-deberta/scripts/train_deberta_local.py': [Errno 2] No such file or directory
/usr/bin/python3: can't open file '/home/user/goemotions-deberta/scripts/train_deberta_local.py': [Errno 2] No such file or directory
E0903 11:24:58.677000 140358872911872 torch/distributed/elastic/multiprocessing/api.py:826] failed (exitcode: 2) local_rank: 0 (pid: 141103) of binary: /usr/bin/python3
Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 7, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/accelerate_cli.py", line 50, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/launch.py", line 1226, in launch_command
    multi_gpu_launcher(args)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/launch.py", line 853, in multi_gpu_launcher
    distrib_run.run(args)
  File "/usr/local/lib/python3.10/dist-packages/to

In [3]:
# Check results
import json
import os

# Ensure we're in the right directory
os.chdir('/home/user/goemotions-deberta')
print(f"📁 Current directory: {os.getcwd()}")

# Check DeBERTa results
deberta_report = "./outputs/deberta/eval_report.json"
if os.path.exists(deberta_report):
    print("🎉 DeBERTa-v3-large training completed!")
    with open(deberta_report, "r") as f:
        rep = json.load(f)
    print("Model:", rep["model"])
    print("F1_micro:", rep["f1_micro"], " F1_macro:", rep["f1_macro"])
    print("F1_micro (t=0.3):", rep.get("f1_micro_t3", "N/A"), " F1_macro (t=0.3):", rep.get("f1_macro_t3", "N/A"))
    print("F1_micro (t=0.5):", rep.get("f1_micro_t5", "N/A"), " F1_macro (t=0.5):", rep.get("f1_macro_t5", "N/A"))
else:
    print("❌ DeBERTa training not completed yet")


📁 Current directory: /home/user/goemotions-deberta
❌ DeBERTa training not completed yet
